In [34]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.model_selection import train_test_split
from ganblr.utils import get_demo_data
import warnings
warnings.filterwarnings('ignore')

df = get_demo_data('car') #Optional: adult or car
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


In [35]:
data = OrdinalEncoder(dtype=int).fit_transform(df)
x, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

In [38]:
from ganblr.ganblr import GANBLR
model = GANBLR()
model.fit(X_train, y_train, k=0, batch_size=32, epochs = 100)

27/27 [==============================] - 1s 654us/step - loss: 1.5148 - accuracy: 0.2465


  0%|          | 0/7 [00:00<?, ?it/s]

27/27 [==============================] - 1s 654us/step - loss: 1.8273 - accuracy: 0.2535


  0%|          | 0/7 [00:00<?, ?it/s]

27/27 [==============================] - 1s 654us/step - loss: 2.1406 - accuracy: 0.2743


...

  0%|          | 0/7 [00:00<?, ?it/s]

27/27 [==============================] - 1s 646us/step - loss: 3.4719 - accuracy: 0.8854


  0%|          | 0/7 [00:00<?, ?it/s]

27/27 [==============================] - 1s 616us/step - loss: 0.6660 - accuracy: 0.8854


  0%|          | 0/7 [00:00<?, ?it/s]

In [39]:
acc_score_lr = model.evaluate(X_test, y_test, model='lr')
acc_score_mlp = model.evaluate(X_test, y_test, model='mlp')
acc_score_rf = model.evaluate(X_test, y_test, model='rf')

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
trtr_score_lr  = LogisticRegression().fit(X_train, y_train).score(X_test, y_test)
trtr_score_rf  = RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test)
trtr_score_mlp = MLPClassifier().fit(X_train, y_train).score(X_test, y_test)

In [41]:
import pandas as pd
df_evaluate = pd.DataFrame([
    ['TSTR', acc_score_lr, acc_score_rf, acc_score_mlp],
    ['TRTR', trtr_score_lr,trtr_score_rf,trtr_score_mlp]
], columns=['Evaluate Item', 'LR', 'RF', 'MLP'])
df_evaluate
#df_evaluate.set_index('Evaluate Item')

,Evaluate Item,LR,RF,MLP
0,TSTR,0.648148,0.776620,0.800926
1,TRTR,0.681713,0.946759,0.869213
